# **Indexación y selección de datos**

**GeoPandas** hereda los métodos de **Pandas** para la **indexación y selección de datos**, lo que permite **extraer subconjuntos** de información según **condiciones tabulares**. Además, incorpora un **método de selección espacial** basado en coordenadas, que facilita la **extracción de datos** cuyas **geometrías intersectan un cuadro delimitador**.

En este artículo, exploraremos en detalle estos métodos de selección y extracción de datos.

Primero, importaremos las librerías necesarias y cargaremos los datos que utilizaremos en este tutorial.

In [1]:
# importar librerías
import numpy as np
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Ruta del archivo
uri=r'D:\Charlie\01_Cartografia\catastro.gpkg'

# Lectura como GeoDataFrame
puertas = gpd.read_file(uri, layer='puertas')

In [3]:
# Visualizar las dos primera filas
puertas.head(2)

,OBJECTID,CODIGOPREDIO,NUMEROLOTE,NUMEROPUERTA,CODIGODISTRITO,CODIGOSEGMENTOVIA,CODIGOMANZANA,CODIGOPUERTA,geometry
0,61,L38,None,842,150132,147461,31856.0,3687,POINT (282306.453 8669914.377)
1,62,L24,None,811A,150132,147899,31856.0,3688,POINT (282245.352 8669863.459)


Veamos las columnas de esta capa:

In [4]:
puertas.columns.values

array(['OBJECTID', 'CODIGOPREDIO', 'NUMEROLOTE', 'NUMEROPUERTA',
       'CODIGODISTRITO', 'CODIGOSEGMENTOVIA', 'CODIGOMANZANA',
       'CODIGOPUERTA', 'geometry'], dtype=object)

## **1. Selección con Operadores de Indexación**

**GeoPandas** permite seleccionar datos mediante los operadores **`[]`** y **`.`**, que facilitan el **acceso a columnas específicas** y la **selección de subconjuntos mediante filtros condicionales**. A continuación, exploramos su aplicación en detalle.

### **1.1. Selección de una columna específica**

Podemos seleccionar una columna específica utilizando el operador **`[]`**, indicando su nombre. También es posible emplear el operador **`.`**, siempre que el nombre de la columna no contenga espacios ni caracteres especiales.

In [5]:
# Seleccionar la columna NUMEROPUERTA usando "[]"
puertas['NUMEROPUERTA'].head()

0     842
1    811A
2     930
3     943
4     971
Name: NUMEROPUERTA, dtype: object

In [6]:
# Seleccionar la columna NUMEROPUERTA usando "."
puertas.NUMEROPUERTA.head()

0     842
1    811A
2     930
3     943
4     971
Name: NUMEROPUERTA, dtype: object

### **1.2. Selección de multiples columnas**

Para seleccionar múltiples columnas, usamos el operador **`[]`** e indicamos una lista con los nombres de las columnas a filtrar:

In [7]:
# Lista con las columnas a seleccionar. Tambien se 
# puede indicar directamente dentro del operador "[]"
cols=['NUMEROPUERTA', 'CODIGODISTRITO', 'geometry']

# Selección de columnas en base a la lista "cols"
puertas[cols].head()

# tambien se puede usar la lista directamente:
# puertas[['NUMEROPUERTA','CODIGODISTRITO','geometry']].head()

,NUMEROPUERTA,CODIGODISTRITO,geometry
0,842,150132,POINT (282306.453 8669914.377)
1,811A,150132,POINT (282245.352 8669863.459)
2,930,150132,POINT (282501.278 8670013.832)
3,943,150132,POINT (282505.574 8669977.579)
4,971,150132,POINT (282560.12 8669989.886)


### **1.3. Selección por condición**

Para filtrar un subconjunto de datos, podemos **aplicar una condición** dentro del operador **`[]`**, seleccionando únicamente las filas que cumplan con el criterio especificado.

In [8]:
# Condición donde el codigo de distrito es "150117"
condicion = puertas.CODIGODISTRITO=='150117'

# Seleccionar solo los registros donde CODIGODISTRITO es '150117'
puertas_olivos = puertas[condicion]

# Imprimir la cantidad de filas y columnas seleccionadas
print(f'Fila y columnas: {puertas_olivos.shape}')

# Visualizar las dos primeras filas:
puertas_olivos.head(2)

# La condición se puede aplicar directamente dentro del operador "[]"
#puertas_olivos = puertas[puertas.CODIGODISTRITO=='150117']

Fila y columnas: (54408, 9)


,OBJECTID,CODIGOPREDIO,NUMEROLOTE,NUMEROPUERTA,CODIGODISTRITO,CODIGOSEGMENTOVIA,CODIGOMANZANA,CODIGOPUERTA,geometry
408797,594697,L18,0,S/N,150117,239440,44601.0,837603,POINT (273398.254 8671700.521)
408798,594698,L01,6,S/N,150117,239995,44593.0,837604,POINT (273837.064 8671477.364)


Mediante los operadores **AND** (`&`) y **OR** (`|`), es posible combinar múltiples condiciones de búsqueda.

In [9]:
# Condición 1: Registros cuyo CODIGODISTRITO es '150117'
conDist = puertas.CODIGODISTRITO=='150117'
# Condición 2: Registros cuyo NUMEROPUERTA es diferente a "S/N"
conPuerta = puertas.NUMEROPUERTA != 'S/N'

# Seleccionando por las condiciones:
puertas_olivos_names = puertas[conDist & conPuerta]

# Imprimir la cantidad de filas y columnas seleccionadas
print(f'Fila y columnas: {puertas_olivos_names.shape}')

# Visualizar las dos primeras filas:
puertas_olivos_names.head(2)

Fila y columnas: (19970, 9)


,OBJECTID,CODIGOPREDIO,NUMEROLOTE,NUMEROPUERTA,CODIGODISTRITO,CODIGOSEGMENTOVIA,CODIGOMANZANA,CODIGOPUERTA,geometry
408847,594747,L13,None,319,150117,239305,44595.0,837653,POINT (273774.795 8671638.546)
408857,594757,L08,None,1888,150117,239193,44602.0,837663,POINT (273334.322 8671831.08)


### **1.3. Integrando selección por condición y columnas**

También es posible unificar la **selección por condición y por columnas** en una sola operación.

In [10]:
# cols: Listado de columnas previamente declaradas
# conDist y conPuerta: Condicions previamente declaradas
puertas_select = puertas[conDist & conPuerta][cols]

# Imprimir la cantidad de filas y columnas seleccionadas
print(f'Fila y columnas: {puertas_select.shape}')

# Visualizar las dos primeras filas:
puertas_select.head(2)

Fila y columnas: (19970, 3)


,NUMEROPUERTA,CODIGODISTRITO,geometry
408847,319,150117,POINT (273774.795 8671638.546)
408857,1888,150117,POINT (273334.322 8671831.08)


## **2. Selección por etiqueta `.loc`**

La selección por etiqueta se realiza mediante el método _Pandas_ **[.loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)**, que permite **acceder a filas y columnas a través de sus nombres de índice**.

**Sintaxis básica**:

```{note}
df.loc[fila, columna]

* **fila**: Etiqueta o rango de etiquetas de las filas a seleccionar.
* **columna**: Etiqueta o rango de etiquetas de las columnas a seleccionar.
```

### **2.1. Selección de filas**

Para seleccionar filas, basta con indicar su nombre, que por defecto es un número entero a menos que se haya especificado otro valor. Si no se indica ninguna columna, se seleccionarán todas por defecto.

Para seleccionar un **única fila** solo indicamos su nombre dentro de **loc**:

In [11]:
puertas.loc[5]

OBJECTID                                                 66
CODIGOPREDIO                                            L17
NUMEROLOTE                                             None
NUMEROPUERTA                                            528
CODIGODISTRITO                                       150132
CODIGOSEGMENTOVIA                                    159355
CODIGOMANZANA                                       31811.0
CODIGOPUERTA                                           3692
geometry             POINT (282541.47200000007 8670064.495)
Name: 5, dtype: object

Para seleccionar **múltiples filas**, se debe proporcionar una lista con sus nombres. Si los indices son valores enteros, es posible aplicar un **slice** para seleccionar un rango de filas. A continuación, exploramos ambos casos:

In [12]:
# Seleccionando un lista de filas
puertas.loc[[5,10,37]]

,OBJECTID,CODIGOPREDIO,NUMEROLOTE,NUMEROPUERTA,CODIGODISTRITO,CODIGOSEGMENTOVIA,CODIGOMANZANA,CODIGOPUERTA,geometry
5,66,L17,None,528,150132,159355,31811.0,3692,POINT (282541.472 8670064.495)
10,71,L15,None,819A,150132,152388,31852.0,3697,POINT (282279.506 8669810.186)
37,98,L10,None,1029B,150132,156930,31818.0,3724,POINT (282732.879 8669959.43)


In [13]:
# Seleccionando filas con un slice 
# (solo si los indices son números enteros)
puertas.loc[40:42]

,OBJECTID,CODIGOPREDIO,NUMEROLOTE,NUMEROPUERTA,CODIGODISTRITO,CODIGOSEGMENTOVIA,CODIGOMANZANA,CODIGOPUERTA,geometry
40,101,L10,None,547,150132,162589,31625.0,3727,POINT (281663.218 8670001.857)
41,102,L17,None,100,150132,155519,31625.0,3728,POINT (281605.177 8669902.147)
42,103,L47,None,S/N,150132,-1,31617.0,3729,POINT (281436.459 8669876.181)


### **2.2. Selección de columnas**

El método **`loc`** también permite seleccionar solo columnas. Para obtener todas las filas, se utiliza **`:`** en la posición correspondiente. Aplicado de esta forma, los resultados obtenidos son equivalentes a los generados mediante los operadores de indexación (`[]`).

Selección de **una única columna**: "NUMEROPUERTA"

In [14]:
puertas.loc[:,'NUMEROPUERTA'].head(3)

0     842
1    811A
2     930
Name: NUMEROPUERTA, dtype: object

Para seleccionar **múltiples columnas**, podemos proporcionar directamente una lista con sus nombres. Sin embargo, para mejorar la legibilidad del código, declararemos una variable cols que contenga dicha lista.

In [15]:
cols= ['NUMEROPUERTA', 'CODIGODISTRITO', 'geometry']
puertas.loc[:, cols].head(3)

,NUMEROPUERTA,CODIGODISTRITO,geometry
0,842,150132,POINT (282306.453 8669914.377)
1,811A,150132,POINT (282245.352 8669863.459)
2,930,150132,POINT (282501.278 8670013.832)


### **2.3. Selección de filas y columnas**

Una de las principales ventajas de **`loc`** es su capacidad para filtrar filas y columnas simultáneamente de forma intuitiva, utilizando etiquetas en ambas dimensiones. Esto facilita la aplicación de **selecciones condicionales** y la **asignación de nuevos valores**.

Selección de **una fila y una columna**:

In [16]:
# Seleccionando un lista de filas
puertas.loc[5, 'NUMEROPUERTA']

'528'

Selección de **multiples filas y columnas**

In [17]:
puertas.loc[[5,6,7,8], ['CODIGOPREDIO','NUMEROPUERTA']]

,CODIGOPREDIO,NUMEROPUERTA
5,L17,528
6,L11,529
7,L10,535
8,L01,273


Selección de **múltiples filas y columnas**, aplicando un slice para las filas:

In [18]:
puertas.loc[5:8, ['CODIGOPREDIO','NUMEROPUERTA']]

,CODIGOPREDIO,NUMEROPUERTA
5,L17,528
6,L11,529
7,L10,535
8,L01,273


### **2.4. Selección por condición**

Con **`loc`**, es posible realizar selecciones condicionales, filtrando filas que cumplen una determinada condición y mostrando únicamente las columnas de interés.

A continuación, realizaremos una selección condicional para mostrar todas las filas donde `NUMEROPUERTA` no tiene nombre, mostrando únicamente las columnas `CODIGOPREDIO y `NUMEROPUERTA`. Aplicaremos estas condiciones directamente:

In [19]:
puertas.loc[puertas.NUMEROPUERTA == 'S/N', ['CODIGOPREDIO','NUMEROPUERTA']].head(3)

,CODIGOPREDIO,NUMEROPUERTA
9,L16,S/N
12,L44,S/N
14,L19,S/N


Al calcular estadísticas de completitud, el valor "S/N" se considera un dato válido. Sin embargo, no representa un número de puerta real, por lo que podemos depurarlo reemplazándolo por NaN. Esto permitirá obtener estadísticas de completitud más precisas.

En el siguiente apartado, aplicaremos esta limpieza asignando valores a una columna específica.

### **2.5. Asignación de nuevo valores**

Una de las ventajas más relevantes de **`loc`** es su capacidad para asignar valores a una o varias columnas, siempre que las filas cumplan una determinada condición.

En el apartado anterior, identificamos filas donde `NUMEROPUERTA` no tiene un valor válido, lo que puede afectar los análisis de completitud. En este ejemplo, reemplazaremos esos valores por **`NaN`** para mejorar la calidad de los datos.

In [20]:
# Reemplazando los NUMEROPUERTA sin nombre por NaN
puertas.loc[puertas.NUMEROPUERTA == 'S/N', 'NUMEROPUERTA'] = np.nan

# Verificando
puertas.loc[puertas.NUMEROPUERTA.isna(), ['CODIGOPREDIO','NUMEROPUERTA']].head(3)

,CODIGOPREDIO,NUMEROPUERTA
9,L16,NaN
12,L44,NaN
14,L19,NaN


Del mismo modo, podemos crear una nueva columna, por ejemplo donde se guarde el dato si el número es par o impar:

In [21]:
# Condicion para evaluar puertas pares
conPares = puertas.NUMEROPUERTA.str.extract(r"(\d+)")[0].str[-1]\
                  .isin([str(i) for i in list(range(0,9,2))])

# Condicion para evaluar puertas impares
conNones = puertas.NUMEROPUERTA.str.extract(r"(\d+)")[0].str[-1]\
                  .isin([str(i) for i in list(range(1,10,2))])

# Aplicando las condiciones para calcular un nuevo campo "Paridad"
puertas.loc[conPares,'paridad'] = 'Pares'
puertas.loc[conNones,'paridad'] = 'Impares'

# Visualizando:
puertas[['CODIGOPREDIO','NUMEROPUERTA','paridad']].head(10)

,CODIGOPREDIO,NUMEROPUERTA,paridad
0,L38,842,Pares
1,L24,811A,Impares
2,L21,930,Pares
3,L13,943,Impares
4,L06,971,Impares
5,L17,528,Pares
6,L11,529,Impares
7,L10,535,Impares
8,L01,273,Impares
9,L16,NaN,NaN


## **3. Selección por posición `.iloc`**

La selección por posición se realiza mediante el método _Pandas_ **[.iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)**, que permite **acceder a filas y columnas mediante índices enteros basados en su posición**.

## **4. Selección por cuadro delimitador `.cx`**